# [비정형 데이터 분석] 가사와 앨범 커버를 이용한 힙합/비힙합 분류

                                                                                                        20142567 최영제

# #0_Bucks Crawling

#### 노래 가사 및 앨범 커버를 통해서 힙합/비힙합을 분류하는 프로젝트입니다.

#### 이 스크립트를 통해서 train data를 긁어왔습니다. 벅스 뮤직에서 긁어왔으며

#### 힙합 최신곡 4000개, 비힙합(발라드/댄스/팝/알앤비/소울) 4000개를 긁어왔습니다.

#### train 데이터를 긁어와 학습을 시켰다면 마지막으로 2018.10.08~2018.10.14 기간 동안

#### Top100곡을 최종 test 데이터로 모델의 일반화 성능을 확인하였으며 긁어오는 과정은 밑에 있습니다.

# #1_Bucks Textmining

#### 이 스크립트로 각종 텍스트 분석을 진행하였습니다.

# #2_Bucks image

#### 앨범 커버를 CNN으로 학습시키는 스크립트 입니다.

# #3_Bucks Testing

#### 위에서 말했던 최종 테스트 성능을 확인하는 스크립트입니다.

In [1]:
import requests
import lxml.html
import time
from random import *
import pandas as pd
import re
from PIL import Image
import matplotlib.pylab as plt
%matplotlib inline

# 제목 뽑아내기

In [2]:
url = 'https://music.bugs.co.kr/genre/kpop/rnh/total?tabtype=2&sort=default&nation=all&page=1'
res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
res.status_code

200

In [3]:
element = lxml.html.fromstring(res.text)
heading = element.cssselect('p.title a')
title = heading[2].text_content()
title

'너나해 (Feat. manu)'

# 가수 뽑아내기

In [4]:
tmp = element.cssselect('p.artist a')
## 원래는 p.artist a로 진행하다가 cover artist와 같이 가수가 아닌 경우는 해당 태그에 포함되어 있지 않아 다음과 같이 변형
tmp = element.cssselect('p.artist')
singer = tmp[0].text_content()
singer

"제이켠(J'Kyun)"

가수 양옆에 붙어있는 공백을 제거한다

In [ ]:
re.sub(r'\s',"",tmp[0].text_content())

# 가사 뽑아내기

In [5]:
tmp = element.cssselect('td a.trackInfo')
lyric_link = tmp[0].attrib['href']
lyric_link

'https://music.bugs.co.kr/track/31328434?wl_ref=list_tr_08'

In [6]:
url = lyric_link
res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
print(res.status_code)

200


In [7]:
element = lxml.html.fromstring(res.text)
tmp = element.cssselect('div.lyricsContainer xmp')
tmp_lyric = tmp[0].text_content()
tmp_lyric

"간을 보지마 우린 막나가니까 뿌릅뿌릅\r\n야 더니 라잎 알바시간 놓쳤냐 야야야\r\n아 저런 형들때매 내가 재밌나\r\n연락해 아이폰 X 배터리 만땅 \r\nFvck I’m 35 새끼들은 말만 많아 말\r\n그래서 우린 계속 이어가 \r\nGRVV Yeah Testify\r\n힙합 찐따들은 배신감\r\n유일하게 다시 치고와\r\n밑바닥을 다시 찍고 Ahk\r\n과소 평가됐지 알아먹냐 니들이 내말을\r\n그럼 평가해봐 내 미래의 가격을 함 띡\r\n삐까뻔쩍한 내 차를 원해\r\n아직 멀었지 평론가들은 여전히 막 짖어\r\n찢어내지 켜니오빠 시절은 왜봐\r\n삐졌네 저 새끼들은 친구가 없나\r\n빙신들은 댓글다는게 직업이냐\r\n떠들거면 니들이 나처럼 함해봐 우\r\nFvck it I’m phenomenon \r\n 내 옷은 절대 못벗기지 마미손\r\n한국힙합 망해라 그래 이 말이 좀\r\n재밌었지 꾸준히들 개지랄이져\r\nFvck it I don’t care\r\nGRVV 우린 채널에있지 \r\n너넨 구경해 이제 내가 끝났데 있지\r\n그래 바깥공기 마셔 대답줄에 임마 \r\nOh I see ooh ha\r\n간을 보지마 우린 막나가니까 뿌릅뿌릅\r\n야 더니 라잎 알바시간 놓쳤냐 야야야\r\n아 저런 형들때매 내가 재밌나\r\n연락해 아이폰 X 배터리 만땅 \r\n간을 보지마 던치 막 나가니 깐\r\n빙신 같은 색기들 싸그리 패버리니 깐\r\n간보는 색기들 어디갓어\r\n간보던 색기들 어디갓어\r\n간보는 색기들 우리들 등장에\r\n꽁무니 빼기가 바빠졌어\r\n개쩌는 집단이 나타났어\r\n쌘척을 해봤자 안무서워\r\n니 멘탈을 부셔서 갈아마시고서\r\nNew school 접수해 한번에 삼켰어 whooo\r\n존만한 애들 비키라고\r\n그만해 키보드 지랄 좀\r\n너네 귀에 묻어있는 피 닦어\r\n천만원짜리 트로필 만져\r\nAY 내 목표는 40 ay\r\n40인치 짜리 바질 허리에ay\r\n걸치고 발을 올려 엑셀\r\n물론 차의 국적은 독일\r\n쉽게 말해

공백을 지워준다

In [8]:
lyric = re.sub(r'\s'," ",tmp_lyric)
lyric

"간을 보지마 우린 막나가니까 뿌릅뿌릅  야 더니 라잎 알바시간 놓쳤냐 야야야  아 저런 형들때매 내가 재밌나  연락해 아이폰 X 배터리 만땅   Fvck I’m 35 새끼들은 말만 많아 말  그래서 우린 계속 이어가   GRVV Yeah Testify  힙합 찐따들은 배신감  유일하게 다시 치고와  밑바닥을 다시 찍고 Ahk  과소 평가됐지 알아먹냐 니들이 내말을  그럼 평가해봐 내 미래의 가격을 함 띡  삐까뻔쩍한 내 차를 원해  아직 멀었지 평론가들은 여전히 막 짖어  찢어내지 켜니오빠 시절은 왜봐  삐졌네 저 새끼들은 친구가 없나  빙신들은 댓글다는게 직업이냐  떠들거면 니들이 나처럼 함해봐 우  Fvck it I’m phenomenon    내 옷은 절대 못벗기지 마미손  한국힙합 망해라 그래 이 말이 좀  재밌었지 꾸준히들 개지랄이져  Fvck it I don’t care  GRVV 우린 채널에있지   너넨 구경해 이제 내가 끝났데 있지  그래 바깥공기 마셔 대답줄에 임마   Oh I see ooh ha  간을 보지마 우린 막나가니까 뿌릅뿌릅  야 더니 라잎 알바시간 놓쳤냐 야야야  아 저런 형들때매 내가 재밌나  연락해 아이폰 X 배터리 만땅   간을 보지마 던치 막 나가니 깐  빙신 같은 색기들 싸그리 패버리니 깐  간보는 색기들 어디갓어  간보던 색기들 어디갓어  간보는 색기들 우리들 등장에  꽁무니 빼기가 바빠졌어  개쩌는 집단이 나타났어  쌘척을 해봤자 안무서워  니 멘탈을 부셔서 갈아마시고서  New school 접수해 한번에 삼켰어 whooo  존만한 애들 비키라고  그만해 키보드 지랄 좀  너네 귀에 묻어있는 피 닦어  천만원짜리 트로필 만져  AY 내 목표는 40 ay  40인치 짜리 바질 허리에ay  걸치고 발을 올려 엑셀  물론 차의 국적은 독일  쉽게 말해 새꺄  일반적인 음악  집어치고 나는 느낌대로 가지  빙신 새끼들 말이 많지 아직  그래서 던치 get'em up   that shit  간을 보지마 우린 막나가니까 뿌릅뿌릅

# 이미지 추출

In [16]:
image_path = '/home/ubunt/3-2/Unstructured/project/data/{}/'.format("hiphop")
element = lxml.html.fromstring(res.text)
images = element.cssselect('li.big a img')
images[0].attrib['src']

'https://image.bugsm.co.kr/album/images/200/202100/20210091.jpg?version=20181123120116.0'

In [17]:
cnt = 0
res = requests.get(images[0].attrib['src'])
with open(image_path + 'cover{}.jpg'.format(cnt), 'wb') as f:
    f.write(res.content)
cnt += 1

# [힙합] 반복문으로 제목, 가수, 링크 추출하기

In [2]:
hiphop_csv_path = '/home/ubunt/3-2/Unstructured/project/data/{}/'.format("hiphop_csv")
else_csv_path = '/home/ubunt/3-2/Unstructured/project/data/{}/'.format("else_csv")

In [13]:
# num = 뽑고자 하는 노래 개수
num = 4000
print('페이지 수 :',int(num/50))

페이지 수 : 80


In [ ]:
titles = []
singers = []
links = []
cnt = 0

for i in range(1,int(num/50)+1) :
    url = 'https://music.bugs.co.kr/genre/kpop/rnh/total?tabtype=2&sort=default&nation=all&page={}'.format(i)
    res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_title = element.cssselect('p.title a')
    raw_singer = element.cssselect('p.artist')
    raw_link = element.cssselect('td a.trackInfo')
    
    for j in range(len(raw_title)) :
        titles.append(raw_title[j].text_content())
        singers.append(re.sub(r'\s',"",raw_singer[j].text_content()))
        links.append(raw_link[j].attrib['href'])
            
    time.sleep(uniform(0, 0.9))
    print(i)

# [힙합] 가사 추출 및 저장

In [15]:
save_time = 0

In [16]:
lyrics = list(range(0, len(links)))

In [ ]:
for i in range(save_time,len(links)) :
    save_time = i
    res = requests.get(links[i], headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_lyrics = element.cssselect('div.lyricsContainer xmp')
    
    if len(raw_lyrics) == 0 :
        lyrics[i] = 'NA'
    else :
        tmp_lyric = raw_lyrics[0].text_content()
        lyrics[i] = re.sub(r'\s'," ",tmp_lyric)
        
    time.sleep(uniform(0, 0.3))
    print(i)

In [18]:
hiphop_csv = pd.DataFrame({'title' : titles,'singer' : singers, 'lyric' : lyrics, 'link' : links})

In [19]:
hiphop_csv.to_csv(hiphop_csv_path + '1124_2000_hiphopcsv4000.csv')

# [힙합] 이미지 추출 및 저장

In [21]:
save_time = 0

In [ ]:
image_path = '/home/ubunt/3-2/Unstructured/project/data/{}/'.format("hiphop")

for i in range(save_time,len(links)) :
    save_time = i
    res = requests.get(links[i], headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_images = element.cssselect('li.big a img')
    
    if len(raw_images) == 0 :
        time.sleep(uniform(0, 0.3))
        continue
    else :
        res = requests.get(raw_images[0].attrib['src'])
        with open(image_path + 'hip_hop_{}.jpg'.format(save_time), 'wb') as f:
            f.write(res.content)
    
    time.sleep(uniform(0, 0.3))
    print(i)

# [발라드/댄스/팝] 반복문으로 제목, 가수, 링크 추출하기

In [65]:
# num = 뽑고자 하는 노래 개수
num = 300
print('페이지 수 :',int(num/50))

페이지 수 : 1


In [66]:
titles = []
singers = []
links = []
cnt = 0

for i in range(1,int(num/50)+1) :
    url = 'https://music.bugs.co.kr/genre/kpop/ballad/total?tabtype=2&sort=default&nation=all&page={}'.format(i)
    res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_title = element.cssselect('p.title a')
    raw_singer = element.cssselect('p.artist')
    raw_link = element.cssselect('td a.trackInfo')
    
    for j in range(len(raw_title)) :
        titles.append(raw_title[j].text_content())
        singers.append(re.sub(r'\s',"",raw_singer[j].text_content()))
        links.append(raw_link[j].attrib['href'])
            
    time.sleep(uniform(0, 0.9))
    print(i)

1


# [발라드/댄스/팝] 가사 추출 및 저장

In [ ]:
save_time = 0

In [ ]:
lyrics = list(range(0, len(links)))

In [ ]:
for i in range(save_time,len(links)) :
    save_time = i
    res = requests.get(links[i], headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_lyrics = element.cssselect('div.lyricsContainer xmp')
    
    if len(raw_lyrics) == 0 :
        lyrics[i] = 'NA'
    else :
        tmp_lyric = raw_lyrics[0].text_content()
        lyrics[i] = re.sub(r'\s'," ",tmp_lyric)
        
    time.sleep(uniform(0, 0.3))
    print(i)

In [ ]:
ballad_csv = pd.DataFrame({'title' : titles,'singer' : singers, 'lyric' : lyrics, 'link' : links})

In [ ]:
ballad_csv.to_csv(else_csv_path + '1123_2000_balladcsv3000.csv')

# [발라드/댄스/팝] 이미지 추출 및 저장

In [ ]:
save_time = 0

In [ ]:
image_path = '/home/ubunt/3-2/Unstructured/project/data/{}/'.format("else")

for i in range(save_time,len(links)) :
    save_time = i
    res = requests.get(links[i], headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_images = element.cssselect('li.big a img')
    
    if len(raw_images) == 0 :
        time.sleep(uniform(0, 0.3))
        continue
    else :
        res = requests.get(raw_images[0].attrib['src'])
        with open(image_path + 'ballad_{}.jpg'.format(save_time), 'wb') as f:
            f.write(res.content)
    
    time.sleep(uniform(0, 0.3))
    print(i)

# [알앤비/소울] 반복문으로 제목, 가수, 링크 추출하기

In [70]:
# num = 뽑고자 하는 노래 개수
num = 300
print('페이지 수 :',int(num/50))

페이지 수 : 1


In [71]:
titles = []
singers = []
links = []
cnt = 0

for i in range(1,int(num/50)+1) :
    url = 'https://music.bugs.co.kr/genre/kpop/rns/total?tabtype=2&sort=default&nation=all&page={}'.format(i)
    res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_title = element.cssselect('p.title a')
    raw_singer = element.cssselect('p.artist')
    raw_link = element.cssselect('td a.trackInfo')
    
    for j in range(len(raw_title)) :
        titles.append(raw_title[j].text_content())
        singers.append(re.sub(r'\s',"",raw_singer[j].text_content()))
        links.append(raw_link[j].attrib['href'])
            
    time.sleep(uniform(0, 0.9))
    print(i)

1


# [알앤비/소울] 가사 추출 및 저장

In [ ]:
lyrics = list(range(0, len(links)))

In [ ]:
for i in range(save_time,len(links)) :
    save_time = i
    res = requests.get(links[i], headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_lyrics = element.cssselect('div.lyricsContainer xmp')
    
    if len(raw_lyrics) == 0 :
        lyrics[i] = 'NA'
    else :
        tmp_lyric = raw_lyrics[0].text_content()
        lyrics[i] = re.sub(r'\s'," ",tmp_lyric)
        
    time.sleep(uniform(0, 0.3))
    print(i)

In [73]:
rnb_csv = pd.DataFrame({'title' : titles,'singer' : singers, 'lyric' : lyrics, 'link' : links})

In [74]:
rnb_csv.to_csv(else_csv_path + '1123_2000_rnbcsv3000.csv')

# [알앤비/소울] 이미지 추출 및 저장

In [ ]:
save_time = 0

In [ ]:
image_path = '/home/ubunt/3-2/Unstructured/project/data/{}/'.format("else")

for i in range(save_time,len(links)) :
    save_time = i
    res = requests.get(links[i], headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    raw_images = element.cssselect('li.big a img')
    
    if len(raw_images) == 0 :
        time.sleep(uniform(0, 0.3))
        continue
    else :
        res = requests.get(raw_images[0].attrib['src'])
        with open(image_path + 'rnb_{}.jpg'.format(save_time), 'wb') as f:
            f.write(res.content)
    
    time.sleep(uniform(0, 0.3))
    print(i)

# 최종 데이터 추출하기

## 제목

In [ ]:
url = 'https://music.bugs.co.kr/chart/track/week/total?chartdate=20181010'
res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
element = lxml.html.fromstring(res.text)
heading = element.cssselect('p.title a')
title = []
for head in heading :
    title.append(head.text_content())

## 가수

In [ ]:
tmp = element.cssselect('p.artist')
singer = []
for sing in tmp :
    singer.append(re.sub(r'\s',"",sing.text_content()))

## 가사

In [ ]:
tmp_link = element.cssselect('td a.trackInfo')
lyric = []
for link in tmp_link :
    url = link.attrib['href']
    res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    tmp = element.cssselect('div.lyricsContainer xmp')
    tmp_lyric = tmp[0].text_content()
    lyric.append(re.sub(r'\s'," ",tmp_lyric))

## 이미지

In [ ]:
image_path = '/home/ubunt/3-2/Unstructured/project/data/final_test/{}/'.format("image")

In [ ]:
cnt = 0
for link in tmp_link :
    url = link.attrib['href']
    res = requests.get(url, headers = {'user-agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:63.0) Gecko/20100101 Firefox/63.0'})
    element = lxml.html.fromstring(res.text)
    images = element.cssselect('li.big a img')

    res = requests.get(images[0].attrib['src'])
    with open(image_path + 'cover{}.jpg'.format(cnt), 'wb') as f:
        f.write(res.content)
        cnt += 1

In [ ]:
label = ["else","hiphop","hiphop","else","else","else","else","hiphop","else","else",
         "else","else","else","else","else","else","else","else","hiphop","hiphop",
         "else","else","else","hiphop","hiphop","else","else","else","else","else",
        "else","hiphop","else","else","else","else","else","hiphop","else","else",
         "else","else","else","else","else","hiphop","else","else","else","else",
         "else","else","else","else","else","else","else","else","hiphop","else",
         "else","else","hiphop","else","else","else","else","else","hiphop", "else",
         "else","else","else","else","else","else","else","else","else","else",
         "else","else","else","else","else","else","else","else","else","else",
         "else","else","else","else","else","else","else","hiphop","else","hiphop"]

In [ ]:
finalTest_csv_path = '/home/ubunt/3-2/Unstructured/project/data/final_test/'

In [ ]:
final_test_csv = pd.DataFrame({'title' : title,'singer' : singer, 'lyric' : lyric, 'label' : label})

In [ ]:
def removeNan(x) :
    print("제거된 NaN 값의 개수 : ",sum(x['lyric'].isna() == True))
    x = x[x['lyric'].isna() == False]
    return(x)

def removeEtc(x) :
    for i in range(x.shape[0]) :
        x['lyric'][i] = re.sub(r'[^a-zA-Z가-힣0-9 ]',"",x['lyric'][i])
        x['lyric'][i] = re.sub(r' {2,}',"",x['lyric'][i])
    return(x)

In [ ]:
final_test_csv = removeNan(final_test_csv)
final_test_csv = removeEtc(final_test_csv)

In [ ]:
final_test_csv.to_csv("final_test.csv")